# Create a new item and upload a file

## Download this notebook rather than running it in Google Colab

This script is from https://docs.figshare.com/#upload_files

In [1]:
import hashlib
import json
import os
import requests
from requests.exceptions import HTTPError

### Set variables
Change the values as appropriate

In [2]:
BASE_URL = 'https://api.figsh.com/v2/{endpoint}' #change this to https://api.figshare.com/v2/{endpoint} if you want to upload to production
TOKEN = 'ENTER TOKEN'
CHUNK_SIZE = 10485760 #about 10MB

In [3]:
FILE_PATH = '../../Sound recordings/user-mtg-16nov2021.m4a' #If the file is in the same folder as this notebook, put the name of the file in quotes
TITLE = 'Uploaded through API' #This is the title of the item you want to upload to.

### Define the functions
These functions create a new item with the title you entered, find your file and divide it into parts, upload each part, and complete the upload.

If you want to add more metadata, add values to the create_article function following the format defined here: https://docs.figsh.com/#private_article_create

In [7]:
#Define functions that split the file, initiate upload, and complete the upload

def raw_issue_request(method, url, data=None, binary=False):
    headers = {'Authorization': 'token ' + TOKEN}
    if data is not None and not binary:
        data = json.dumps(data)
    response = requests.request(method, url, headers=headers, data=data)
    try:
        response.raise_for_status()
        try:
            data = json.loads(response.content)
        except ValueError:
            data = response.content
    except HTTPError as error:
        print('Caught an HTTPError: {}'.format(error.message))
        print('Body:\n', response.content)
        raise

    return data


def issue_request(method, endpoint, *args, **kwargs):
    return raw_issue_request(method, BASE_URL.format(endpoint=endpoint), *args, **kwargs)


def list_articles():
    result = issue_request('GET', 'account/articles')
    print('Listing current articles:')
    if result:
        for item in result:
            print(u'  {url} - {title}'.format(**item))
    else:
        print('  No articles.')



def create_article(title):
    data = {
        'title': TITLE  # You may add any other information about the article here as you wish.
    }
    result = issue_request('POST', 'account/articles', data=data)
    print('Created article:', result['location'], '\n')

    result = raw_issue_request('GET', result['location'])

    return result['id']


def list_files_of_article(article_id):
    result = issue_request('GET', 'account/articles/{}/files'.format(article_id))
    print('Listing files for article {}:'.format(article_id))
    if result:
        for item in result:
            print('  {id} - {name}'.format(**item))
    else:
        print('  No files.')

    print


def get_file_check_data(file_name):
    with open(file_name, 'rb') as fin:
        md5 = hashlib.md5()
        size = 0
        data = fin.read(CHUNK_SIZE)
        while data:
            size += len(data)
            md5.update(data)
            data = fin.read(CHUNK_SIZE)
        return md5.hexdigest(), size


def initiate_new_upload(article_id, file_name):
    endpoint = 'account/articles/{}/files'
    endpoint = endpoint.format(article_id)

    md5, size = get_file_check_data(file_name)
    data = {'name': os.path.basename(file_name),
            'md5': md5,
            'size': size}

    result = issue_request('POST', endpoint, data=data)
    print('Initiated file upload:', result['location'], '\n')

    result = raw_issue_request('GET', result['location'])

    return result


def complete_upload(article_id, file_id):
    issue_request('POST', 'account/articles/{}/files/{}'.format(article_id, file_id))


def upload_parts(file_info):
    url = '{upload_url}'.format(**file_info)
    result = raw_issue_request('GET', url)

    print('Uploading parts:')
    with open(FILE_PATH, 'rb') as fin:
        for part in result['parts']:
            upload_part(file_info, fin, part)


def upload_part(file_info, stream, part):
    udata = file_info.copy()
    udata.update(part)
    url = '{upload_url}/{partNo}'.format(**udata)

    stream.seek(part['startOffset'])
    data = stream.read(part['endOffset'] - part['startOffset'] + 1)

    raw_issue_request('PUT', url, data=data, binary=True)
    print('  Uploaded part {partNo} from {startOffset} to {endOffset}'.format(**part))


def main():
    # We first create the article
    list_articles()
    article_id = create_article(TITLE)
    list_articles() #Prints the list again and you should see your new item at the top of the list
    list_files_of_article(article_id)

    # Then we upload the file.
    file_info = initiate_new_upload(article_id, FILE_PATH)
    # Until here we used the figshare API; following lines use the figshare upload service API.
    upload_parts(file_info)
    # We return to the figshare API to complete the file upload process.
    complete_upload(article_id, file_info['id'])
    list_files_of_article(article_id)

### Initiate and complete the upload
This will print the status of operations as they occur.

In [8]:
#Run this to upload the file.
if __name__ == '__main__':
    main()

Listing current articles:
  https://api.figsh.com/v2/account/articles/7341492 - Test article title
  https://api.figsh.com/v2/account/articles/7334522 - Untitled Item
  https://api.figsh.com/v2/account/articles/7183896 - Validation of a Postextubation Dysphagia Screening Tool for Patients After Prolonged Endotracheal Intubation
  https://api.figsh.com/v2/account/articles/7183894 - Open access journal publication
  https://api.figsh.com/v2/account/articles/7147054 - Full embargo
  https://api.figsh.com/v2/account/articles/7124066 - Box plot of heights for treatment and control in the study
  https://api.figsh.com/v2/account/articles/7039568 - Dataset for my research
  https://api.figsh.com/v2/account/articles/6954644 - Item in Review
  https://api.figsh.com/v2/account/articles/6954642 - Filming Locations for Animal House
  https://api.figsh.com/v2/account/articles/5794944 - Newsletter Template with Great Preview
Created article: https://api.figsh.com/v2/account/articles/7369646 

Listin